In [ ]:
PROJECT_ID = 'razekmh-angola'

In [ ]:
# install libraries
! pip install -q earthengine-api geemap

In [2]:
# import libraries
import ee
import geemap

In [ ]:
# authenticate and initialize Earth Engine API
try:
    ee.authenticate()
    ee.Initialize(project=PROJECT_ID)
    print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
    print('The Earth Engine package failed to initialize!')
